In [26]:
# imports
from pydantic import BaseModel,Field

from langgraph.graph import StateGraph, END, START

from typing import Dict

In [27]:
class DiscountState(BaseModel):
    total : float = Field(...,ge=0,description="Total price of the order")
    discount_rate: float = Field(0.0,ge=0,le=1,description="Discount rate to apply to the order")
    final_price:float = Field(0.0,ge=0,description="Final price of the order after discount")
    

In [28]:
def check_eligibility(state: DiscountState) -> Dict:
    total = state.total
    if total > 1000:
        return {'discount_rate': 0.20}
    elif total > 500:
        return {'discount_rate': 0.10}
    else:
        return {'discount_rate': 0.00}

In [29]:
def apply_discount(state: DiscountState) -> Dict:
    total = state.total
    rate = state.discount_rate
    final = total *( 1- rate)
    return {'final_price': final}

In [30]:
def no_discount(state: DiscountState)-> Dict:
    return {'final_price': state.total}

In [31]:
def router_discount(state: DiscountState) -> Dict:
    if state.discount_rate > 0:
        return "apply"
    else:
        return "no"

In [32]:
graph = StateGraph(DiscountState)

graph.add_node("apply_discount", apply_discount)
graph.add_node("check_eligibility", check_eligibility)
graph.add_node("no_discount", no_discount)

graph.add_edge(START, "check_eligibility")

#graph.add_edge("check_eligibility","apply_discount")
#graph.add_edge("check_eligibility","no_discount")
graph.add_conditional_edges("check_eligibility", router_discount,{
    "apply": "apply_discount",
    "no": "no_discount"
})


graph.add_edge("apply_discount", END)

graph.add_edge("no_discount", END)



In [33]:
workflow = graph.compile()

In [34]:
response = workflow.invoke({"total": 300})

In [35]:
print(response)

{'total': 300, 'discount_rate': 0.0, 'final_price': 300.0}
